In [1]:
import requests
import json
from pprint import pprint


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain_community.utilities import SerpAPIWrapper

In [92]:
from dotenv import load_dotenv
load_dotenv()

True

In [18]:
search=SerpAPIWrapper()
response=search.run("What is the email address of company Where U Elevate?")

In [17]:
help(search)

Help on SerpAPIWrapper in module langchain_community.utilities.serpapi object:

class SerpAPIWrapper(pydantic.main.BaseModel)
 |  SerpAPIWrapper(*, search_engine: Any = None, params: dict = {'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key: Optional[str] = None, aiosession: Optional[aiohttp.client.ClientSession] = None) -> None
 |  
 |  Wrapper around SerpAPI.
 |  
 |  To use, you should have the ``google-search-results`` python package installed,
 |  and the environment variable ``SERPAPI_API_KEY`` set with your API key, or pass
 |  `serpapi_api_key` as a named parameter to the constructor.
 |  
 |  Example:
 |      .. code-block:: python
 |  
 |          from langchain_community.utilities import SerpAPIWrapper
 |          serpapi = SerpAPIWrapper()
 |  
 |  Method resolution order:
 |      SerpAPIWrapper
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  async

In [19]:
print(response)

['Contact Email care@wuelev8.tech ; Phone Number +91 9560900306.', "Here's how to get in touch with the Elevate team for more information or to schedule a demo for our best-in-class benefit account platform.", 'It also allows users to post jobs. Company Details. Website: whereuelevate.com; Social: Twitter Facebook; Email ID: *** ...', 'Where U Elevate, Bareilly. 271 likes · 4 talking about this. Where U Elevate is a talent management platform with 5M+ Impressions, 200k+ Users, 3000+...', '1st Floor, 27 – 33. Bethnal Green Rd, London, E16LA ... hellouk@weareelevate.global · +44 (0)207 183 0303 ... Network ... Australia ... Netherlands ... China ... Poland ...', 'What Information do we collect? Personal Details: Full name, Email address, Phone number, Address, Date of birth, Gender, Marital Status; Professional Details: ...', 'Email: US_sales@elevatesaas.com Phone: +1 (949) 528-8333 Fax: +1 (800) 679-4617 Address: 3990 Westerly Suite 270 Newport Beach', 'New business inquiries, please ca

[


In [23]:
# Specifying parameters relative to the Indian region
params = {
    "engine": "google",
    "gl": "in",
    "google_domain":"google.co.in",
    "hl": "en",
}

In [45]:
import inspect
print(inspect.signature(SerpAPIWrapper))

(*, search_engine: Any = None, params: dict = {'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key: Optional[str] = None, aiosession: Optional[aiohttp.client.ClientSession] = None) -> None


In [46]:
print(SerpAPIWrapper.__doc__)

Wrapper around SerpAPI.

    To use, you should have the ``google-search-results`` python package installed,
    and the environment variable ``SERPAPI_API_KEY`` set with your API key, or pass
    `serpapi_api_key` as a named parameter to the constructor.

    Example:
        .. code-block:: python

            from langchain_community.utilities import SerpAPIWrapper
            serpapi = SerpAPIWrapper()
    


In [30]:
search=SerpAPIWrapper(params=params)
response=search.run("What is the email address of company Where U Elevate?")

In [32]:
print(response)

['Where U Elevate c/o WUELEV8 Innovation Services Pvt. Ltd. F-13, Sweet Home Colony, Shastri Nagar, Bareilly - 243122 (UP), India. For Inquiries. Sales and ...', 'It also allows users to post jobs. Company Details. Website: whereuelevate.com; Social: Twitter Facebook; Email ID: *** ...', 'Where U Elevate is an AI-driven platform nurturing Open Innovation in India, linking Industry and Academia through Hackathons, Next-Gen Hiring, Contractual ...', 'Contact Email care@wuelev8.tech ; Phone Number +91 9560900306.', 'Where U Elevate, Bareilly. 271 likes · 3 talking about this. Where U Elevate is a talent management platform with 5M+ Impressions, 200k+ Users, 3000+...', 'Find the best suited opportunity to showcase your innovative skills with available jobs, hackathons, industry led Open Innovation programs and career mentoring.', '1st Floor, 27 – 33. Bethnal Green Rd, London, E16LA ... hellouk@weareelevate.global · +44 (0)207 183 0303 ... Network ... Australia ... Netherlands ... China ...

In [36]:
# According to the displayed response I think this is the splitting pattern, so splitting like this
results = response.split("', ")
for i, result in enumerate(results, start=1):
    print(f"Result {i}: {result}\n")

Result 1: ['Where U Elevate c/o WUELEV8 Innovation Services Pvt. Ltd. F-13, Sweet Home Colony, Shastri Nagar, Bareilly - 243122 (UP), India. For Inquiries. Sales and ...

Result 2: 'It also allows users to post jobs. Company Details. Website: whereuelevate.com; Social: Twitter Facebook; Email ID: *** ...

Result 3: 'Where U Elevate is an AI-driven platform nurturing Open Innovation in India, linking Industry and Academia through Hackathons, Next-Gen Hiring, Contractual ...

Result 4: 'Contact Email care@wuelev8.tech ; Phone Number +91 9560900306.

Result 5: 'Where U Elevate, Bareilly. 271 likes · 3 talking about this. Where U Elevate is a talent management platform with 5M+ Impressions, 200k+ Users, 3000+...

Result 6: 'Find the best suited opportunity to showcase your innovative skills with available jobs, hackathons, industry led Open Innovation programs and career mentoring.

Result 7: '1st Floor, 27 – 33. Bethnal Green Rd, London, E16LA ... hellouk@weareelevate.global · +44 (0)207 

In [43]:
# testing with google gemini 
# zero shot react description
# with no serpapi params lets see

llm=ChatGoogleGenerativeAI(temperature=0.2,model="gemini-1.5-flash-latest")
tools=load_tools(["serpapi"],llm=llm)
agent=initialize_agent(tools,llm,agent="zero-shot-react-description",verbose=True)
agent.run("What is the email address of company Where U Elevate ?")



> Entering new AgentExecutor chain...
Thought: I need to find the email address of Where U Elevate.  A web search should help.

Action: Search
Action Input: "Where U Elevate contact email"

Observation: For general or account inquiries, please email support@elevateapp.com.
Thought:Thought: I now know the final answer.
Final Answer: support@elevateapp.com


> Finished chain.


'support@elevateapp.com'

In [50]:
print(help(load_tools))

Help on function load_tools in module langchain_community.agent_toolkits.load_tools:

load_tools(tool_names: List[str], llm: Optional[langchain_core.language_models.base.BaseLanguageModel] = None, callbacks: Union[List[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, allow_dangerous_tools: bool = False, **kwargs: Any) -> List[langchain_core.tools.BaseTool]
    Load tools based on their name.
    
    Tools allow agents to interact with various resources and services like
    APIs, databases, file systems, etc.
    
    Please scope the permissions of each tools to the minimum required for the
    application.
    
    For example, if an application only needs to read from a database,
    the database tool should not be given write permissions. Moreover
    consider scoping the permissions to only allow accessing specific
    tables and impose user-level quota for limiting resource usage.
    
    Please read the AP

In [56]:
from langchain.agents import Tool

In [55]:

llm = ChatGoogleGenerativeAI(temperature=0.2, model="gemini-1.5-flash-latest")
params = {
    "engine": "google",
    "gl": "in",
    "google_domain": "google.co.in",
    "hl": "en",
}

search = SerpAPIWrapper(params=params)

# Load tools, passing the search tool into the agent
tools = [Tool(name="serpapi", func=search.run, description="Search for information online")]

agent = initialize_agent(tools, llm, agent_type="zero-shot-react-description", verbose=True)

response = agent.run("What is the email address of company Where U Elevate?")
print(response)




> Entering new AgentExecutor chain...
Thought: I need to find the email address of Where U Elevate.  I can try searching for it online using SerpApi.

Action: serpapi
Action Input: query="Where U Elevate email address"

Observation: ['Where U Elevate c/o WUELEV8 Innovation Services Pvt. Ltd. F-13, Sweet Home Colony, Shastri Nagar, Bareilly - 243122 (UP), India. For Inquiries. Sales ...', 'Find the best suited opportunity to showcase your innovative skills with available jobs, hackathons, industry led Open Innovation programs and career mentoring.', 'Where U Elevate is an AI-driven platform nurturing Open Innovation in India, linking Industry and Academia through Hackathons, Next-Gen Hiring, Contractual ...', 'Where U Elevate is working on creating the next generation innovative workforce which is agile, adaptable, and innovative by leveraging cutting edge ...', "Let's connect and explore how we can help you build a future-ready team fast and efficiently. Book a demo or reach out to u

- > Directly using Google Gemini with this is talking a lots of time
- > Maybe I should just try using Groq (maybe it doesnt) or I should pass the output of tools directly using a prompt template I guess, let's see what works the best

- > Another Problem, Response provided by Google is not always correct here, (Also gemini taking a lots of time here , lets redefine our approach) as initially first api call-> email yielded, second call-> not yieleded


#### Modifying approach to use TavilySearchResults

In [59]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(max_results=2)
search.invoke("what is the email address of company Where U Elevate ?")

[{'url': 'https://www.crunchbase.com/organization/where-u-elevate',
  'content': 'Legal Name WUELEV8 Innovation Services Private Limited. Company Type For Profit. Contact Email care@wuelev8.tech. Phone Number +91 9560900306. Where U Elevate is an AI-driven platform nurturing "Open Innovation" in India, linking Industry and Academia through Hackathons, Next-Gen Hiring, Contractual Research, and Technology Transfers.'},
 {'url': 'https://contactout.com/company/Where-U-Elevate-75986',
  'content': "Get details for Where U Elevate's 10 employees, email format for wuelev8.tech and phone numbers. Where U Elevate is a platform that empowers engineers to engage in ongoing innovation journeys, taking their careers to new heights. Our platform offers undergrads and working professionals recognized for their contributions, providing access to top job recommendations, hiring challenges, and"}]

In [60]:
# I guess this is looking fine, as far as things are concerned
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(max_results=2)
search.invoke("I lost my facebook account. How I can mail them? Have they provided an gmail address ?")

[{'url': 'https://www.pcmag.com/how-to/how-to-recover-a-facebook-account',
  'content': "If you're locked out of your account because you've been hacked or you've just noticed activity on your account that's suspicious (changes to your personal info that you didn't make"},
 {'url': 'https://en-gb.facebook.com/help/mobile-basic/231208473756221/',
  'content': "Recover an old Facebook account that you can't log in to. Go to the profile of the account that you'd like to recover. Tap More below the cover photo, then select Find support or report. Tap Something else, then tap Submit. Tap Recover this account, then tap Submit and follow the steps. You can learn more about hacked accounts."}]

In [61]:
search=SerpAPIWrapper(params=params)
response=search.run("I lost my facebook account. How I can mail them? Have they provided an gmail address ?")

In [63]:
print(response)

['Try these steps to recover your account: Try using a different mobile phone number or email address, which you might have forgotten are associated with your ...', "If you can't receive emails or text messages from us, maybe because you lost or upgraded your phone or your email account was hacked, learn how to recover ...", 'You will need to undertake the Gmail Account Recovery ("AR") procedure. If you have already done this, we need you to try once again.', 'To recover your Facebook account using your Gmail, go to the Facebook login page, click on "Forgotten account?" and follow the prompts. You\'ll ...', 'Account Recovery ("AR") is the ONLY way to recover the account. If you are unwilling to undertake AR, Google will refuse all further assistance.', "Lost access to your Facebook account because you can't access your email or phone number? Don't worry! In this video, I'll show you how to ...", "To recover an old account: 1. Tap and enter the profile name. 2. Tap the name of the profi

In [67]:
print(type(response))

<class 'str'>


In [69]:
results = response.split("', ")
for i, result in enumerate(results, start=1):
    print(f"Result {i}: {result}\n")

Result 1: ['Try these steps to recover your account: Try using a different mobile phone number or email address, which you might have forgotten are associated with your ...

Result 2: "If you can't receive emails or text messages from us, maybe because you lost or upgraded your phone or your email account was hacked, learn how to recover ...", 'You will need to undertake the Gmail Account Recovery ("AR") procedure. If you have already done this, we need you to try once again.

Result 3: 'To recover your Facebook account using your Gmail, go to the Facebook login page, click on "Forgotten account?" and follow the prompts. You\'ll ...

Result 4: 'Account Recovery ("AR") is the ONLY way to recover the account. If you are unwilling to undertake AR, Google will refuse all further assistance.

Result 5: "Lost access to your Facebook account because you can't access your email or phone number? Don't worry! In this video, I'll show you how to ...", "To recover an old account: 1. Tap and enter 

- > In this no one was actually able to provide gmail address


In [74]:
search = TavilySearchResults(max_results=2)
search.invoke("What is the email address of Zomato ?")

[{'url': 'https://www.grievanceofficer.com/grievance-officer-zomato/',
  'content': 'Email address: legal@zomato.com'},
 {'url': 'https://www.easyleadz.com/company/zomato',
  'content': 'Zomato is a private company and Zomato is classified as a Non Government company. Zomato is a Consumer Services based company and has headquarters in Gurugram, Haryana, India. Zomato has 7174 employees working in various departments like Sales, Marketing, HR, Finance & Operations. It was founded in 2008. Zomato specialises in Restaurant Search & Discovery, Table Reservations, Restaurant Pos'}]

In [84]:
search = TavilySearchResults(max_results=2)
search.invoke("Mail address of Zomato site:zomato.com")

[{'url': 'https://www.zomato.com/policies/terms-of-service/',
  'content': 'Email address: nodal@zomato.com. For Customers residing in India, please note, in compliance with The Information Technology Act, 2000 and the rules made thereunder, as well as The Consumer Protection Act 2019, and the rules made thereunder, the grievance redressal mechanism, including the contact details of the Grievance Officer and Nodal'},
 {'url': 'https://www.zomato.com/investor-relations/resources',
  'content': "By using this site you agree to Zomato's use of cookies to give you a personalised experience. Please read the cookie policy for more information or to delete/block them. Accept ... Investor Grievance Email. delhi@linkintime.co.in. Website. www.linkintime.co.in. Contact Person. Mr. Vishal Dixit. Zomato Limited. CIN: L93030DL2010PLC198141"}]

In [76]:
search.invoke("""site:zomato.com "support" "@gmail.com""")

[{'url': 'https://www.zomato.com/contact',
  'content': "Click on the 'Support' or 'Online ordering help' section in your app to connect to our customer support team. By using this site you agree to Zomato's use of cookies to give you a personalised experience. Please read the cookie policy for more information or to delete/block them. Accept"},
 {'url': 'https://www.zomato.com/mumbai/contact',
  'content': "Click on the 'Support' or 'Online ordering help' section in your app to connect to our customer support team. Select Country. India. UAE. India. Select Language. English. About Zomato. Who We Are. Blog. Work With Us. Investor Relations. Report Fraud. Press Kit. Contact Us. Zomaverse. Zomato. Blinkit. District."}]

In [77]:
search.invoke("Mail address of Zomato Customer Support Team")

[{'url': 'https://www.reddit.com/r/delhi/comments/16tsy1e/how_do_i_actually_get_in_touch_with_a_zomato/',
  'content': 'o get a in refund Zomato, you can follow these steps: Contact Zomato Customer Support:08409-850-997 visit the Zomato website. Log in to your account using your registered email ID and password. Go to the "My Orders" section. Select the order for which you want a refund. Click on the "Support" or "Help" option associated with the order.'},
 {'url': 'https://www.consumercomplaints.in/zomato-b115724/customer-care',
  'content': 'Zomato Customer Care Contacts  Zomato Phone Numbers Zomato Website Zomato Head Office Address Zomato Social Media Zomato Company Representatives Additional Zomato contact information United States United Kindgom Mr. Deepinder Goyal Chief Executive Officer, Co-Founder and Director Mr. Pankaj Chaddah Chief Operating Officer Mr. Gunjan Patidar Zomato Overview Zomato ranks as the #1 most complained about among Food. officially connected with Zomato. 

In [71]:
search=SerpAPIWrapper(params=params)
response=search.run("What is the mail address of Facebook Support?")

In [78]:
search=SerpAPIWrapper(params=params)
response=search.run("""site:zomato.com "support" "@gmail.com""")

In [79]:
print(response)

["Issue with your live order? Click on the 'Support' or 'Online ordering help' section in your app to connect to our customer support team.", 'All Zomato employees in client facing roles including but not limited to Sales, Sales Support ... Please email legal@zomato.com if you witness any behavior ...', 'For any help or queries pertaining to Event(s), you may reach out to us through chat support or write to us at eventsupport@zomato.com. Previous. « Go out ...', 'Use this page to report a serious incident or accident only. For order related queries, please use our chat support.', '... Email to Collab - foodiesatc@gmail.com Zomato Connoisseur ⚓ ... With @patcummins30 giving him at support at the other end, Australia managed ...', '... gmail.com . Why Book with KBG Trips? . - Expert travel guidance - Personalized support - Seamless booking experience . Secure Your Spot Today!', "What's the one thing that really worked well for you? Ease of finding a restaurant/dish. Food quantity. Custom

In [80]:
search=SerpAPIWrapper(params=params)
response=search.run("""site:whereuelevate.com "support" "@gmail.com""")

In [86]:
search=SerpAPIWrapper(params=params)
response=search.run("""Mail address of Zomato site:zomato.com""")
response

'["Issue with your live order? Click on the \'Support\' or \'Online ordering help\' section in your app to connect to our customer support team.", "Reach out to us by emailing at companysecretary@zomato.com, and we\'ll get back to you. Contact Us. Disclosures under Regulation 46 of the LODR.", \'If you observe any such practice, report it to help@zomato.com. Appropriate action will be taken against the user or employee if evidence can be provided.\', \'Tell us what you think about Zomato. Name Email Phone number (optional ) Message Guidelines Please be brutal and tell us more of what we suck at, than what we ...\', \'Please use this form only for reporting potential frauds. For order or other general queries contact us here. Zomato logo.\', \'If you have questions, or need assistance, please contact us at merchant@zomato.com. Will I be able to install the App on Windows and BlackBerry devices? Sorry, ...\', \'... to deliver your order from the restaurant to your delivery location ... m

In [87]:
print(response)

["Issue with your live order? Click on the 'Support' or 'Online ordering help' section in your app to connect to our customer support team.", "Reach out to us by emailing at companysecretary@zomato.com, and we'll get back to you. Contact Us. Disclosures under Regulation 46 of the LODR.", 'If you observe any such practice, report it to help@zomato.com. Appropriate action will be taken against the user or employee if evidence can be provided.', 'Tell us what you think about Zomato. Name Email Phone number (optional ) Message Guidelines Please be brutal and tell us more of what we suck at, than what we ...', 'Please use this form only for reporting potential frauds. For order or other general queries contact us here. Zomato logo.', 'If you have questions, or need assistance, please contact us at merchant@zomato.com. Will I be able to install the App on Windows and BlackBerry devices? Sorry, ...', '... to deliver your order from the restaurant to your delivery location ... mail address, ad

In [88]:
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.agents import initialize_agent, AgentType
from langchain_community.chat_models import ChatOpenAI
from langchain.tools.tavily_search import TavilySearchResults

In [90]:
lm =ChatGroq(model="llama-3.1-70b-versatile", temperature=0.7)
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search)

# initialize the agent
agent_chain = initialize_agent(
    [tavily_tool],
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

# run the agent
agent_chain.run("What happened in the latest Dehradun car accident of Innova")
    



> Entering new AgentExecutor chain...
Thought:I need to use a search engine to find information about a recent car accident involving an Innova in Dehradun.

Action:
```json
{
  "action": "tavily_search_results_json",
  "action_input": {
    "query": {
      "title": "Dehradun Innova Car Accident",
      "description": "Recent car accident in Dehradun involving an Innova.  Looking for details on the accident.",
      "type": "string"
    }
  }
}
```


ValidationError: 1 validation error for TavilyInput
query
  str type expected (type=type_error.str)

In [91]:
lm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000161BE9C2B90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000161BE9CC690>, model_name='llama-3.1-70b-versatile', groq_api_key=SecretStr('**********'))